In [ ]:
#mount qcow2
sudo modprobe nbd
sudo qemu-nbd -c /dev/nbd0 /var/lib/libvirt/images/W10-clone.qcow2
sudo partx -l /dev/nbd0
sudo mkdir -p /mnt/ESP
sudo mount /dev/nbd0p1 /mnt/ESP


In [ ]:
#AGRUBPARTWIN10FROMUBUNTU

#!/etc/bin/bash

set -k
sudo killall gparted
usbmountdir="/mnt/usb"
sudo mkdir -p $usbmountdir
#letter="b"
#disk="/dev/sd$letter"
disk=$(sudo fdisk -l | grep 238.49 | grep dev | awk '{print $2}' | tr -d ':')
#disk="/dev/nbd0"
#sudo gparted
#sudo blkid
#sudo fdisk -l | grep '^Disk'
for i in {1..5} ; do sudo umount $disk$i ; done
sudo wipefs -a $disk

#W10GPTUEFI
sudo wipefs -a $disk
sudo mkdir -p /mnt/{usb,efi}
sudo parted --script $disk \
    mklabel gpt \
    mkpart ESP fat32 2048s 413695s \
        name 1 '"EFI system partition"' \
        set 1 boot on \
        set 1 esp on \
    mkpart primary fat32 413696s 446696s \
        name 2 '"Microsoft reserved partition"' \
        set 2 msftres on \
    mkpart primary ntfs 446697s 256GB \
        name 3 Windows
sudo killall gparted
sudo gparted
sudo mkfs.vfat -F32 ${disk}1
sudo mkfs.vfat -F32 ${disk}2
sudo mkfs.ntfs -Q -L "Windows" ${disk}3

#W10GPTUEFI
sudo wipefs -a $disk
sudo mkdir -p /mnt/{usb,efi}
sudo parted --script $disk \
    mklabel gpt \
    mkpart ESP fat32 2048s 413695s \
        name 1 '"EFI system partition"' \
        set 1 boot on \
        set 1 esp on \
    mkpart primary fat32 413696s 446696s \
        name 2 '"Microsoft reserved partition"' \
        set 2 msftres on \
    mkpart primary ntfs 446697s 256GB \
        name 3 Windows
sudo killall gparted
sudo gparted
sudo mkfs.vfat -F32 ${disk}p1
sudo mkfs.vfat -F32 ${disk}p2
sudo mkfs.ntfs -Q -L "Windows" ${disk}p3

#GPTUEFIHYBRIDAIOBOOT
sudo mkdir -p /mnt/{usb,efi}
sudo parted --script $disk \
    mklabel gpt \
    mkpart primary fat32 2048s 4095s \
        name 1 BIOS \
        set 1 bios_grub on \
    mkpart ESP fat32 4096s 413695s \
        name 2 EFI \
        set 2 esp on \
    mkpart primary fat32 413696s 2100000s \
        name 3 GRUB \
        set 3 msftdata on \
    mkpart primary ntfs 2100001s 256GB \
        name 4 Windows
sudo killall gparted
sudo gparted
    mkpart primary ntfs 200GB 250GB \
        name 5 WTGSTOR \
    mkpart primary fat32 250GB 251GB \
        name 6 TEMPBOOT


sudo mkfs.vfat -F32 ${disk}2
sudo mkfs.vfat -F32 ${disk}3
sudo mkfs.ntfs -Q -L "Windows" ${disk}4
#
sudo mkfs.ntfs -Q -L "WTGSTORAGE" ${disk}5
sudo mkfs.vfat -F32 -n "TEMPBOOT" ${disk}6

#sudo mount ${disk}2 /mnt/efi && \
sudo mount ${disk}3 $usbmountdir




#GPTUEFI(FOR DEAKIN)
#mklabel msdos BIOS
sudo mkdir -p $usbmountdir
sudo parted --script $disk \
    mklabel gpt \
    mkpart primary fat32 1MiB 1GB \
 mkpart primary ntfs 1GB 200GB \
 mkpart primary ntfs 200GB 250GB \
 mkpart primary fat32 250GB 251GB
 #\ mkpart primary ext4 200GB 300GB \
 # name 3 UBUNTU \
sudo umount ${disk}1
sudo umount ${disk}2
sudo mkfs.vfat -F32 -n "GRUB" ${disk}1
sudo mkfs.ntfs -Q -L "WTG" ${disk}2
sudo mkfs.ntfs -Q -L "WTGSTORAGE" ${disk}3
sudo mkfs.vfat -F32 -n "TEMPBOOT" ${disk}4

#sudo mkfs -t ntfs "${disk}"*1
#sudo mkfs -t ntfs ${disk}2



#LABEL_OLD
#echo mtools_skip_check=1 >> ~/.mtoolsrc ;\
#sudo mlabel -i ${disk}1 ::GRUB ;\
#sudo mlabel -i ${disk}2 ::WTG ;\

sudo umount ${disk}1
sudo umount ${disk}2
sudo umount ${disk}3
sudo umount ${disk}4
sudo umount ${disk}5
sudo umount ${disk}6

#ISODIR
disk=$(sudo fdisk -l | grep 238.49 | grep dev | awk '{print $2}' | tr -d ':')
windownlink=""
isoname="Windows10_InsiderPreview_Client_x64_en-us_20175"
#dir="/home/$USER/Downloads"
dir="/run/media/$USER/SED/VBOX/ISOS"
exdir="$dir/$isoname"
isodir="$dir/$isoname.iso"
wimname="install.wim"
wimdir="$exdir/$wimname"
#sudo mkdir -p $exdir
###7z e "$isodir" -o"$exdir" -y

#installwim
wiminfo "$wimdir"
#yay -S ntfs-3g-wimlib-patched #FIX
sudo wimapply "$wimdir" 6 ${disk}3 --check #W10Pro
#sudo wimapply "$wimdir" 10 ${disk}3 --check #W10ProW
#sudo wimapply "$wimdir" 1 ${disk}3 --check #W10H
#sudo wimapply "$wimdir" 8 ${disk}3 --check #W10ProE
#sudo rm -rf $exdir
#sudo virsh start W10
#https://www.tenforums.com/tutorials/131765-apply-unattended-answer-file-windows-10-install-media.html
#https://www.tenforums.com/installation-upgrade/127414-automate-just-oobe-setup-pass-windows-10-autounattend-xml-2.html
#https://www.rizaumami.com/2017/07/04/install-windows-7-from-within-debian-sid/
#https://onedrive.live.com/?authkey=%21ANwv8oCSQQlH7tM&cid=FCDCA7D2E5A3FC02&id=FCDCA7D2E5A3FC02%21218388&parId=FCDCA7D2E5A3FC02%21218387&o=OneUp
#https://drive.google.com/file/d/0B30UcQryc941Mk1BVmVEUmNDS00/view
#https://docs.microsoft.com/en-us/previous-versions/windows/it-pro/windows-8.1-and-8/hh824950(v=win.10)?redirectedfrom=MSDN
#/run/media/$USER/Windows/Windows/Panther/Unattend
cd /run/media/$USER/Windows/Windows
mkdir -p /run/media/$USER/Windows/Windows/Panther/Unattend
cp /run/media/$USER/SED/VBOX/GitHub/os_from_scratch/W10_autounattend.xml /run/media/$USER/Windows/Windows/Panther/Unattend/unattend.xml
#
cp /run/media/$USER/SED/VBOX/GitHub/os_from_scratch/W10_init.ps1 /run/media/$USER/Windows/W10_init.ps1
cp -r /run/media/$USER/SED/VBOX/GitHub/os_from_scratch/ChocolateyPackageListBackup /run/media/$USER/Windows/ChocolateyPackageListBackup
mkdir -p /run/media/$USER/Windows/Users/Old
cp -r /run/media/$USER/SED/VBOX/SP3/* /run/media/$USER/Windows/Users/Old

sudo umount -l /run/media/$USER/Windows
sudo virsh start W10-Fix
sudo umount -l /run/media/$USER/Windows
sudo virsh start W10
#sudo cp /run/media/$USER/SED/VBOX/GitHub/os_from_scratch/autounattend.xml /run/media/$USER/Windows/unattend.xml
#sudo cp /run/media/$USER/SED/VBOX/GitHub/os_from_scratch/autounattend.xml /run/media/$USER/Windows/Windows/System32/Sysprep/unattend.xml
#%windir%\System32\Sysprep\sysprep.exe /oobe /reboot /unattend:C:\unattend.xml

##OOBE
#https://community.spiceworks.com/topic/1949582-setting-up-sysprep-to-run-script-after-logon
<FirstLogonCommands>
                <SynchronousCommand wcm:action="add">
                    <CommandLine>powershell.exe -File C:\script.ps1</CommandLine>
                    <Order>1</Order>
                </SynchronousCommand>
            </FirstLogonCommands>

#GRUBMENU
wimstuffdir="/home/$USER/Documents/Docs/mlinuxproject/wimstuffs"
usbmountdir="/mnt/usb"
stormountdir="/mnt/stor"
letter="b"
#letter="a"
grubnumber="1"
wtgnumber="2"
stornumber="3"

#GPTUEFIHYBRID
grubnumber="3"
wtgnumber="4"
stornumber="5"

disk="/dev/sd$letter"
sudo mkdir -p /mnt/{usb}
sudo mount ${disk}${grubnumber} $usbmountdir
installgrubpc
installgrubefi
usbgrubcfgdir="$usbmountdir/boot/grub/grub.cfg"
MBothgrub "3" "5" "$usbgrubcfgdir"
sudo blkid /dev/sd$letter$wtgnumber
sudo gedit $usbgrubcfgdir

sudo mkdir -p $stormountdir
sudo mount ${disk}${stornumber} $stormountdir
cp -r $wimstuffdir $stormountdir
sudo umount /mnt/{usb,stor}









#grep error in ubuntu
#allerror in ubuntu
#wimapply install
unmountcurdisk() {
export disk=$usbpart &&\
sudo umount /mnt/{usb,efi} ;\
sudo umount /media/$userid/$curlabel ;\
return 0
}
installgrubpc() {
sudo grub-install \
    --target=i386-pc \
    --boot-directory=$usbmountdir/boot \
    --recheck \
    $disk
return 0
}
installgrubefi() {
sudo grub-install \
    --target=x86_64-efi \
    --efi-directory=$usbmountdir \
    --boot-directory=$usbmountdir/boot \
    --removable \
    --recheck
return 0
}

REINSTALLTOOLS() {
git clone git://git.code.sf.net/p/ntfs-3g/ntfs-3g
cd ntfs-3g
git checkout edge

sudo apt-get purge ntfs-3g -y
sudo apt-get install autoconf autogen autotools -y
sudo ./autogen.sh
./configure
make
sudo make install



##################### Basic
sudo apt-get install openssh-server
sudo apt-get install mc
sudo apt install net-tools
apt-get install git
apt-get install parted
apt-get install build-essential

##################### Requirements

sudo apt-get install autotools-dev autoconf libtool pkg-config libxml2-dev libfuse-dev libgcrypt20-dev linux-source
#linux-source is a requirement!

##################### Build

cd /var/tmp/
#git clone git://git.code.sf.net/p/ntfs-3g/ntfs-3g
cd ntfs-3g
git checkout edge

git reset --hard 67feb2
./autogen.sh
./configure

sudo make
sudo make install

cd ..

sudo wget https://wimlib.net/downloads/wimlib-1.12.0.tar.gz
tar -xvzf wimlib-1.12.0.tar.gz

cd wimlib-1.12.0

./configure --without-fuse --prefix=/usr --build=i686-linux
./configure --without-fuse --prefix=/usr --build=x86_64-linux
make
make install

##################### test
parted /dev/sdb mklabel msdos
mount -a
parted /dev/sdb mkpart primary ntfs 0 100%
mkfs.ntfs -f /dev/sdb1
wimlib-imagex apply /home/andi/install.wim 2 /dev/sdb1
}


MBothgrub() {
grub1=$(cat <<\EOF
# The file must match what we use for the `.iso` file.
EOF
)
grub2=$(cat <<EOF
#search --set=root --file /DEBIAN_CUSTOM_ISO

insmod all_video

set default="9"
set default="$1"
set timeout=$2
EOF
)
grub3=$(cat <<\EOF
#BIOS
menuentry "Ubuntu GUI from RAM" {
linux /casper/vmlinuz.efi boot=casper quiet splash ignore_uuid toram nomodeset
#Ubuntu in safe mode
#linux /casper/vmlinuz.efi boot=casper xforcevesa quiet splash ignore_uuid
#all_generic_ide pci=nommconf ide=nodma acpi=off
initrd /casper/initrd.lz
#initrd /casper/ori2initrd.lz
#initrd /casper/oriinitrd.lz
}

menuentry "Ubuntu CLI from RAM" {
linux /casper/vmlinuz.efi boot=casper systemd.unit=multi-user.target quiet splash ignore_uuid toram nomodeset
initrd /casper/initrd.lz
}

menuentry "Ubuntu GUI persistent mode" {
linux /casper/vmlinuz.efi boot=casper persistent quiet splash ignore_uuid
initrd /casper/initrd.lz
}

set check0="(hd0,msdos2)"
set check1="(hd1,msdos2)"
set check2="(hd2,msdos2)"
menuentry "W10Legacy" {
insmod fat
insmod ntfs
insmod ntldr
insmod part_gpt
insmod part_msdos
insmod search_fs_uuid
insmod chain

if [ -f $check0/Windows/Boot/PCAT/bootmgr ]; then
set root=$check0
elif [ -f $check1/Windows/Boot/PCAT/bootmgr ]; then
set root=$check1
elif [ -f $check2/Windows/Boot/PCAT/bootmgr ]; then
set root=$check2

else
echo "NOT FOUND"
sleep 10
reboot
fi

echo "$root"
sleep 2

#search --no-floppy --set=root --fs-uuid AE10175B10172A3B
search --no-floppy --fs-uuid --set=root 08480D5E08480D5E
#nltdr (${root})/Windows/Boot/PCAT/bootmgr
chainloader (${root})/Windows/Boot/PCAT/bootmgr
#chainloader +1
boot
#nltdr (${root})/bootmgr (winreenv)
}

menuentry "Check Disk for Defects" {
linux /casper/vmlinuz.efi boot=casper integrity-check quiet splash ignore_uuid
initrd /casper/initrd.lz
}

menuentry "Memory Test" {
linux16 /boot/memtest86+.bin
}

menuentry "Boot from the first hard disk" {
set root=(hd0)
chainloader +1
}

#UEFI
insmod chain
set check0="(hd0,msdos2)"
set check1="(hd1,msdos2)"
set check2="(hd2,msdos2)"
menuentry "CHECK" {
if [ -f $check0/casper/filesystem.squashfs ]; then
set root=$check0
elif [ -f $check1/casper/filesystem.squashfs ]; then
set root=$check1
elif [ -f $check2/casper/filesystem.squashfs ]; then
set root=$check2

else
echo "NOT FOUND"
sleep 10
reboot
fi

echo "$root"
sleep 2
configfile /boot/grub/grub.cfg
#set prefix=($root)/boot/grub
#insmod normal
#normal
}

menuentry "Boot DN HDD" {
set prefix=(hd0,msdos1)/boot/grub
insmod normal
normal
}

menuentry "W10LUH(BCD-UEFI)" --class windows --class os {
  insmod fat
  insmod ntfs
  #insmod ntldr
  insmod part_gpt
  insmod part_msdos
  insmod search_fs_uuid
  insmod chain
  #set root="hd0,msdos1"
  if [ x$feature_platform_search_hint = xy ]; then
      search --no-floppy --fs-uuid --set=root AE10175B10172A3B
      search --no-floppy --fs-uuid --set=root 08480D5E08480D5E
  else
      search --no-floppy --fs-uuid --set=root AE10175B10172A3B
      search --no-floppy --fs-uuid --set=root 08480D5E08480D5E
  fi
  #parttool ${root} hidden-
  #drivemap -s (hd0) ${root}
  #chainloader +1
  #Legacy
  #nltdr (${root})/Windows/Boot/PCAT/bootmgr
  #chainloader (${root})/Windows/Boot/PCAT/bootmgr
  #boot
  #EFI
  #chainloader (${root})/Windows/Boot/EFI/bootmgfw.efi
  #chainloader (${root})/EFI/Microsoft/Boot/bootmgfw.efi
  chainloader +1
}

menuentry "WinRE10" --class windows --class os {
insmod fat
insmod ntfs
insmod part_gpt
insmod part_msdos
search --no-floppy --set=root --fs-uuid AE10175B10172A3B
search --no-floppy --fs-uuid --set=root 08480D5E08480D5E
nltdr (${root})/bootmgr
boot
}
#menuentry "Debian Live" {
# linux /vmlinuz boot=live quiet nomodeset
# initrd /initrd
#}
##chainloader (${root})/EFI/Microsoft/Boot/bootmgfw.efi
##chainloader (${root})/Windows/Boot/EFI/bootmgfw.efi
##chainloader (${root})/Windows/Boot/EFI/bootmgr.efi
##chainloader (${root})/Boot/BCD
##chainloader (${root})/bootmgr
##chainloader (${root})/efi/Microsoft/Boot/bootmgfw.efi
##--hint-bios=hd0,msdos2 --hint-efi=hd0,msdos2 --hint-baremetal=ahci0,msdos2
##sudo blkid /dev/sdb1
EOF
)
echo "$grub1
$grub2
$grub3" | sudo tee $3
}



In [ ]:
Custom USB
ubuntu-gnome-16.04.3-desktop-amd64.iso
ubuntu-18.04-desktop-amd64.iso
NVIDIA-Linux-x86_64-390.25.run
amdgpu-pro-17.40-492261.tar.xz
xmrig
xmr-stak
xmirig-nvidia

W10
monero-gui-win-x64-v0.11.1.0
ccminer-cryptonight-master.zip
xmr-stak-cpu-win64.zip
monerospelunker_v01.zip
MinGW-master.zip

xmr-stak-cpu-win64
xmr-stak-master
sgminer-gm-master
MinerGate-cli-4.04-win64
Claymore CryptoNote GPU Miner


In [ ]:
USBLINUX

sudo apt-get update

wget https://download.teamviewer.com/download/linux/teamviewer_amd64.deb
sudo apt install ./teamviewer_amd64.deb -y

sudo apt-get install npm nodejs -y
bash <(curl -sL https://raw.githubusercontent.com/node-red/linux-installers/master/deb/update-nodejs-and-nodered)
sudo systemctl enable nodered.service
node-red-start

sudo apt-get install -y anki openvpn

sudo add-apt-repository ppa:webupd8team/tor-browser -y

sudo apt-get update

sudo apt-get install tor-browser
